<a href="https://colab.research.google.com/github/chihyanghsu0805/machine_learning/blob/tutorials/tutorials/keras/knowledge_distillation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook re-implements the tutorial on https://keras.io/examples/vision/knowledge_distillation/.

The original paper is https://arxiv.org/abs/1503.02531.

First, a `teacher` model is trained.

Then, knowledge is distilled to the `student` model by training with a combined weighted loss function, 

1.   The first objective function is the cross entropy with the soft targets and this cross entropy is computed using the same high temperature in the softmax of the distilled model as was used for generating the soft targets from the cumbersome model. 

2.   The second objective function is the cross entropy with the correct labels. This is computed using exactly the same logits in softmax of the distilled model but at a temperature of 1. We found that the best results were generally obtained by using a condiderably lower weight on the second objective function. 



# Knowledge Distillation with Teacher Student network

## Load Libraries

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np

## Prepare the dataset

In [2]:
# Prepare the train and test dataset.
batch_size = 64
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

# Normalize data
x_train = x_train.astype("float32") / 255.0
x_train = np.reshape(x_train, (-1, 28, 28, 1))

x_test = x_test.astype("float32") / 255.0
x_test = np.reshape(x_test, (-1, 28, 28, 1))

## Distiller

In [3]:
class Distiller(keras.Model):
    def __init__(self, student, teacher):
        super(Distiller, self).__init__()
        self.teacher = teacher
        self.student = student

    def compile(
        self,
        optimizer,
        metrics,
        student_loss_fn,
        distillation_loss_fn,
        alpha=0.1,
        temperature=3,
    ):
        super(Distiller, self).compile(optimizer=optimizer, metrics=metrics)
        self.student_loss_fn = student_loss_fn
        self.distillation_loss_fn = distillation_loss_fn
        self.alpha = alpha
        self.temperature = temperature

    def train_step(self, data):
        x, y = data
        teacher_predictions = self.teacher(x, training=False)

        #   One way to do this is to use the correct labels to modify the soft
        #   targets, but we found that a better way is to simply use a weighted
        #   average of two different objective functions. The first objective
        #   function is the cross entropy with the soft targets and this
        #   cross entropy is computed using the same high temperature in the
        #   softmax of the distilled model as was used for generating the soft
        #   targets from the cumbersome model. The second objective function is
        #   the cross entropy with the correct labels. This is computed using
        #   exactly the same logits in softmax of the distilled model but at a
        #   temperature of 1. We found that the best results were generally
        #   obtained by using a considerably lower weight on the second
        #   objective function. 

        with tf.GradientTape() as tape:
            student_predictions = self.student(x, training=True)

            student_loss = self.student_loss_fn(y, student_predictions)
            distillation_loss = self.distillation_loss_fn(
                tf.nn.softmax(teacher_predictions / self.temperature, axis=1),
                tf.nn.softmax(student_predictions / self.temperature, axis=1),
            )
            loss = self.alpha * student_loss + (1 - self.alpha) * distillation_loss

        trainable_vars = self.student.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)

        self.optimizer.apply_gradients(zip(gradients, trainable_vars))

        self.compiled_metrics.update_state(y, student_predictions)

        results = {m.name: m.result() for m in self.metrics}
        #results.update(
        #    {"student_loss": student_loss, "distillation_loss": distillation_loss}
        #)
        return results

    def test_step(self, data):
        x, y = data

        y_prediction = self.student(x, training=False)

        student_loss = self.student_loss_fn(y, y_prediction)

        self.compiled_metrics.update_state(y, y_prediction)

        results = {m.name: m.result() for m in self.metrics}
        #results.update({"student_loss": student_loss})
        return results


## Teacher

In [4]:
# Create the teacher
teacher = keras.Sequential(
    [
        keras.Input(shape=(28, 28, 1)),
        layers.Conv2D(256, (3, 3), strides=(2, 2), padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.MaxPooling2D(pool_size=(2, 2), strides=(1, 1), padding="same"),
        layers.Conv2D(512, (3, 3), strides=(2, 2), padding="same"),
        layers.Flatten(),
        layers.Dropout(0.5),
        layers.Dense(10),
    ],
    name="teacher",
)

## Student

In [5]:
# Create the student
student = keras.Sequential(
    [
        keras.Input(shape=(28, 28, 1)),
        layers.Conv2D(16, (3, 3), strides=(2, 2), padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.MaxPooling2D(pool_size=(2, 2), strides=(1, 1), padding="same"),
        layers.Conv2D(32, (3, 3), strides=(2, 2), padding="same"),
        layers.Flatten(),
        layers.Dense(10),
    ],
    name="student",
)

# Clone student for later comparison
student_scratch = keras.models.clone_model(student)

## Train the teacher


In [6]:
teacher.compile(
    optimizer=keras.optimizers.Adam(),
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[keras.metrics.SparseCategoricalAccuracy()],
)

teacher.fit(x_train, y_train, epochs=5)
teacher.evaluate(x_test, y_test) # Loss, Accuracy

Epoch 1/5
1875/1875 [==============================] - 20s 9ms/step - loss: 0.1518 - sparse_categorical_accuracy: 0.9538
Epoch 2/5
1875/1875 [==============================] - 13s 7ms/step - loss: 0.1045 - sparse_categorical_accuracy: 0.9702
Epoch 3/5
1875/1875 [==============================] - 13s 7ms/step - loss: 0.0988 - sparse_categorical_accuracy: 0.9726
Epoch 4/5
1875/1875 [==============================] - 13s 7ms/step - loss: 0.0985 - sparse_categorical_accuracy: 0.9739
Epoch 5/5
313/313 [==============================] - 1s 4ms/step - loss: 0.1158 - sparse_categorical_accuracy: 0.9743


[0.11584722995758057, 0.9743000268936157]

## Distill teacher to student


In [7]:
distiller = Distiller(student=student, teacher=teacher)
distiller.compile(
    optimizer=keras.optimizers.Adam(),
    metrics=[keras.metrics.SparseCategoricalAccuracy()],
    student_loss_fn=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    distillation_loss_fn=keras.losses.KLDivergence(),
    alpha=0.1,
    temperature=10,
)

distiller.fit(x_train, y_train, epochs=3)
distiller.evaluate(x_test, y_test)

Epoch 1/3
1875/1875 [==============================] - 7s 3ms/step - sparse_categorical_accuracy: 0.9171
Epoch 2/3
1875/1875 [==============================] - 6s 3ms/step - sparse_categorical_accuracy: 0.9673
Epoch 3/3
313/313 [==============================] - 1s 2ms/step - sparse_categorical_accuracy: 0.9766


0.9765999913215637

## Train student from scratch for comparison

In [8]:
# Train student as doen usually
student_scratch.compile(
    optimizer=keras.optimizers.Adam(),
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[keras.metrics.SparseCategoricalAccuracy()],
)

# Train and evaluate student trained from scratch.
student_scratch.fit(x_train, y_train, epochs=3)
student_scratch.evaluate(x_test, y_test)

Epoch 1/3
1875/1875 [==============================] - 5s 3ms/step - loss: 0.2231 - sparse_categorical_accuracy: 0.9341
Epoch 2/3
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0905 - sparse_categorical_accuracy: 0.9725
Epoch 3/3
313/313 [==============================] - 1s 2ms/step - loss: 0.0739 - sparse_categorical_accuracy: 0.9758


[0.07391741871833801, 0.9757999777793884]